In [9]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
import torch
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa", use_fast=False)
model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa", return_dict=False)

from sentence_transformers import SentenceTransformer, util
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')

from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])
import src.clean_dataset as clean

import time
import pickle5 as pickle

Downloading:   0%|          | 0.00/866 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [2]:
#import data:
df = pd.read_csv('../data/raw/arxiv_results.csv')
df

,Unnamed: 0,title,summary,authors,url
0,0,"Taking Ethics, Fairness, and Bias Seriously in...","This paper highlights an important, if under-e...","['Robert Soden', 'Dennis Wagenaar', 'Dave Luo'...",http://arxiv.org/abs/1912.05538v2
1,1,A Survey on the Role of Wireless Sensor Networ...,Extreme events and disasters resulting from cl...,"['Ahsan Adeel', 'Mandar Gogate', 'Saadullah Fa...",http://arxiv.org/abs/1909.10353v1
2,2,A Natural Disasters Index,"Natural disasters, such as tornadoes, floods, ...","['Thilini V. Mahanama', 'Abootaleb Shirvani']",http://arxiv.org/abs/2008.03672v1
3,3,Sistem Pengambilan Keputusan Penanganan Bencan...,After Aceh's quake many earthquakes have struc...,['H. L. H Spits Warnars'],http://arxiv.org/abs/1006.1704v2
4,4,CNN-Based Semantic Change Detection in Satelli...,Timely disaster risk management requires accur...,"['Ananya Gupta', 'Elisabeth Welburn', 'Simon W...",http://arxiv.org/abs/2006.05589v1
...,...,...,...,...,...
14995,14995,Reduce The Wastage of Data During Movement in ...,In this research paper so as to handle Data in...,"['Ahmed Mateen', 'Lareab Chaudhary']",http://arxiv.org/abs/1612.08702v1
14996,14996,Paving the Way to Smart Micro Energy Internet:...,The energy internet is one of the most promisi...,"['Shengwei Mei', 'Rui Li', 'Xiaodai Xue', 'Yin...",http://arxiv.org/abs/1612.09500v3
14997,14997,A Non-Cooperative Game Theoretical Approach Fo...,Power management is one of the vital issue in ...,"['R. Valli', 'P. Dananjayan']",http://arxiv.org/abs/1007.5168v1
14998,14998,Secured Data Consistency and Storage Way in Un...,It is very challenging part to keep safely all...,['C. Dinesh'],http://arxiv.org/abs/1111.2412v1


In [5]:
df['summary_clean'] = df['summary'].astype(str).apply(clean.basic)
wrapped, splitted = clean.split_at_length(df, 'summary_clean', 512, PIMS_ID = False)

In [6]:
splitted = pd.concat([pd.Series(row['title'], row['wrapped'].split("; "), )              
                            for _, row in wrapped.iterrows()]).reset_index()
splitted = splitted.rename(columns={"index": "text", 0: "title"})

In [7]:
passages = splitted.text.tolist()
passage_id = splitted.title.tolist()

In [8]:
passages

['this paper highlights an important if underexamined set of questions about the deployment of machine learning technologies in the field of disaster risk management drm while emerging tools show promising capacity to support scientific efforts to better understand and mitigate the threats posed by disasters and climate change our field must undertake a much more careful assessment of the potential negative impacts that machine learning technologies may create we also argue that attention to these issues in',
 'the context of machine learning affords the opportunity to have discussions about potential ethics bias and fairness concerns within disaster data more broadly in what follows we first describe some of the uses and potential benefits of machinelearning technology in disaster risk management we then draw on research from other fields to speculate about potential negative impacts finally we outline a research agenda for how our disaster risk management can begin to take these issu